In [1]:
%reload_ext autoreload
%autoreload 2
from SymptomPreprocessing import *
from SpatialPreprocessing import *
import Preprocessing as P
import Cluster
import Metrics
import re
import RPorts
import Utils
from AucComp import delong
from warnings import simplefilter

from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, GradientBoostingRegressor, AdaBoostRegressor, RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.decomposition import PCA
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_selection import f_classif, f_regression, mutual_info_regression, mutual_info_classif
from sklearn.metrics import accuracy_score,f1_score, auc, precision_recall_fscore_support, matthews_corrcoef, roc_auc_score, recall_score, precision_score, roc_curve
from datetime import datetime
import statsmodels.api as sm
from scipy.stats import chi2
import seaborn as sns
import joblib

simplefilter(action='ignore')
pd.set_option('display.max_rows', 1000)

In [2]:
def timestamp():
    return datetime.now().strftime('%Y%d%H%M%S%f')
for i in range(10):
    print(timestamp())
    
def get_all_dvh(ct, key='V'):
    return sorted([col for col in ct.df if re.match(key+'\d+',col) is not None], key = lambda x: int(x[1:]) )


202306152606310239
202306152606310280
202306152606310294
202306152606310305
202306152606310317
202306152606310329
202306152606310339
202306152606310348
202306152606310357
202306152606310418


In [3]:
rds = RadDataset()
rds.get_value_array(rds.dvh_df.columns,as_df=True)

spellchecking...
renaming things
filtering pateints
adding nan values for missing organs
adding histograms
176.0 91.77084266874341 406.1887464658043
264.0 109.22534647050192 407.1108350538819


,Esophagus_max_dose,Esophagus_D5,Esophagus_D45,Esophagus_V50,Esophagus_D80,Esophagus_V75,Esophagus_D90,Esophagus_V40,Esophagus_min_dose,Esophagus_V15,...,Glottic_Area_D10,Glottic_Area_V10,Glottic_Area_D60,Glottic_Area_D30,Glottic_Area_V20,Glottic_Area_V35,Glottic_Area_D25,Glottic_Area_V30,Glottic_Area_V25,Glottic_Area_D70
id,,,,,,,,,,,,,,,,,,,,,
2.0,58.27,53.218750,38.437500,14.367188,29.187500,0.0,13.140625,39.000000,2.97,88.250000,...,39.218750,100.000000,20.171875,29.046875,61.093750,17.515625,30.906250,27.234375,40.843750,18.656250
6.0,50.88,46.125000,38.625000,0.177246,6.699219,0.0,3.509766,32.437500,1.84,75.375000,...,39.968750,100.000000,31.640625,34.312500,100.000000,23.296875,34.656250,83.000000,100.000000,31.093750
7.0,43.62,38.687500,30.843750,0.000000,5.000000,0.0,3.210938,1.930664,1.00,64.750000,...,30.656250,100.000000,28.703125,29.734375,100.000000,0.000000,29.906250,22.828125,100.000000,28.390625
8.0,34.01,32.781250,24.875000,0.000000,6.691406,0.0,6.078125,0.000000,3.52,55.343750,...,27.687500,67.687500,11.890625,15.023438,19.156250,0.000000,15.312500,0.000000,19.156250,9.812500
9.0,59.75,54.468750,35.125000,12.492188,22.515625,0.0,18.312500,32.968750,13.58,96.437500,...,27.296875,50.843750,6.519531,20.046875,30.218750,0.000000,21.531250,4.316406,15.109375,6.218750
10.0,51.81,45.718750,33.968750,0.477051,7.660156,0.0,5.300781,26.921875,3.07,68.937500,...,58.125000,100.000000,39.031250,51.125000,100.000000,72.125000,53.656250,88.875000,99.500000,35.718750
11.0,29.00,26.812500,9.101562,0.000000,6.730469,0.0,6.390625,0.000000,5.94,34.531250,...,19.421875,79.187500,11.070312,16.093750,7.738281,0.000000,18.234375,4.761719,4.761719,10.750000
12.0,6.10,3.900391,1.450195,0.000000,0.700195,0.0,0.479980,0.000000,1.00,0.000000,...,22.703125,31.890625,2.199219,10.726562,12.937500,0.981934,12.781250,3.755859,7.914062,1.940430
13.0,43.43,42.656250,39.437500,0.000000,14.031250,0.0,3.849609,38.781250,1.00,79.625000,...,21.390625,56.156250,9.679688,14.132812,13.515625,0.000000,15.046875,0.000000,4.503906,8.757812


In [50]:
dose_df = P.rolled_up_dvh_valdf(rds)
dose_df.index = dose_df.index.astype('int')
dose_df

,D10,D15,D2,D20,D25,D30,D35,D40,D45,D5,...,V60,V65,V70,V75,V80,level_0,max_dose,mean_dose,min_dose,volume
id,,,,,,,,,,,,,,,,,,,,,
2,"[51.625, 26.890625, 56.1875, 53.625, 53.75, 57...","[49.78125, 26.515625, 54.6875, 52.21875, 52.96...","[54.53125, 28.1875, 59.84375, 57.40625, 55.937...","[47.09375, 25.875, 53.09375, 50.875, 52.4375, ...","[44.4375, 25.453125, 51.375, 49.75, 52.03125, ...","[42.53125, 25.078125, 49.71875, 48.6875, 51.65...","[41.03125, 24.703125, 48.03125, 47.75, 51.3125...","[39.78125, 24.375, 46.46875, 46.625, 51.0, 51....","[38.4375, 24.125, 44.875, 45.6875, 50.5625, 51...","[53.21875, 27.4375, 58.28125, 55.5, 54.90625, ...",...,"[0.0, 0.0, 1.673828125, 0.5244140625, 0.0, 6.2...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.1484375, 0.0, 53.2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 38.40625, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[9.0, 38.0, 3.0, 4.0, 8.0, 7.0, 19.0, 20.0, 5....","[58.27, 30.08, 62.72, 60.34, 57.47, 68.42, 59....","[35.72055395268319, 23.816881203970524, 36.247...","[2.97, 17.68, 3.41, 13.08, 40.7, 25.79, 28.34,...","[14279.47998046875, 12866.363525390625, 7139.7..."
6,"[44.96875, 26.40625, 61.71875, 56.78125, 55.18...","[44.03125, 25.84375, 58.75, 55.9375, 52.40625,...","[48.0625, 28.03125, 65.5625, 58.3125, 61.28125...","[42.78125, 25.40625, 56.71875, 54.875, 50.4687...","[41.4375, 25.03125, 55.46875, 53.625, 49.5625,...","[40.375, 24.65625, 53.4375, 52.375, 49.0, 50.0...","[39.59375, 24.3125, 51.65625, 51.125, 47.8125,...","[39.09375, 24.0, 50.09375, 49.84375, 46.5625, ...","[38.625, 23.671875, 48.53125, 48.75, 45.59375,...","[46.125, 27.1875, 63.8125, 57.5, 58.375, 58.31...",...,"[0.0, 0.0, 12.4296875, 0.0, 3.88671875, 2.2148...","[0.0, 0.0, 2.63671875, 0.0, 0.0, 0.0, 6.867187...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.80810546875, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[49.0, 78.0, 43.0, 44.0, 48.0, 47.0, 59.0, 60....","[50.88, 30.84, 68.96, 59.66, 64.43, 63.04, 71....","[30.499019492025983, 22.78990464240902, 32.607...","[1.84, 13.69, 98.0, 13.56, 35.83, 27.59, 32.93...","[6974.945068359375, 16417.694091796875, 6562.9..."
7,"[37.625, 37.15625, 66.3125, 60.3125, 49.4375, ...","[36.5625, 36.75, 64.0625, 59.96875, 48.375, 49...","[39.96875, 38.0625, 71.8125, 60.90625, 52.1562...","[35.25, 36.34375, 62.71875, 59.71875, 47.34375...","[34.0625, 35.90625, 61.53125, 59.40625, 46.562...","[33.125, 35.5, 60.34375, 59.1875, 45.6875, 43....","[32.28125, 35.03125, 59.03125, 58.90625, 44.75...","[31.609375, 34.625, 57.71875, 58.625, 43.96875...","[30.84375, 34.1875, 56.40625, 58.3125, 43.1875...","[38.6875, 37.625, 71.0625, 60.625, 50.75, 53.8...",...,"[0.0, 0.0, 31.4375, 14.78125, 0.0, 0.0, 6.6875...","[0.0, 0.0, 12.2734375, 0.0, 0.0, 0.0, 0.835937...","[0.0, 0.0, 6.8203125, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[83.0, 115.0, 105.0, 93.0, 82.0, 81.0, 89.0, 1...","[43.62, 38.64, 72.16, 61.26, 55.52, 58.77, 67....","[22.765716284275324, 33.629034598214275, 55.53...","[1.0, 21.14, 36.25, 41.79, 34.59, 29.69, 36.68...","[14724.42626953125, 14765.625, 6645.3552246093..."
8,"[32.15625, 32.0625, 65.375, 56.6875, 22.5, 31....","[31.703125, 31.109375, 63.34375, 55.75, 21.312...","[33.4375, 34.09375, 68.75, 58.28125, 33.6875, ...","[31.203125, 30.25, 61.34375, 54.5625, 17.84375...","[30.609375, 29.40625, 60.03125, 52.375, 14.367...","[29.9375, 28.390625, 59.5, 50.5625, 13.140625,...","[28.96875, 27.453125, 58.96875, 49.46875, 11.7...","[27.1875, 26.46875, 58.53125, 48.59375, 11.203...","[24.875, 25.453125, 58.125, 48.0, 10.9609375, ...","[32.78125, 33.125, 67.375, 57.65625, 25.3125, ...",...,"[0.0, 0.0, 25.578125, 0.0, 0.0, 0.0, 12.851562...","[0.0, 0.0, 10.78125, 0.0, 0.0, 0.0, 0.0, 37.40...","[0.0, 0.0, 0.1573486328125, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...

In [75]:
def format_dose_df(df,
               features=None,
                organ_subset=None,
                normalize=True,
               ):
    if features is None:
        features=['V35','V40','V45','V50','V55','V60','V65']
    if organ_subset is None:
        organ_subset = Const.organ_list[:]
    organ_positions = [Const.organ_list.index(o) for o in organ_subset]
    vals = np.stack(df[features].apply(lambda x: np.stack([np.array([ii[i] for i in organ_positions]).astype(float) for ii in x]).ravel(),axis=1).values)
    if normalize:
        vals = (vals - vals.mean(axis=0))/(vals.std(axis=0) + .01)
    return vals

def get_cluster_closure(fit_df,
                         clusterer = None,
                         features=None,
                         reducer=None,
                         organ_subset=None,
                         normalize = True,
                         prefix='',
                         n_clusters = 4,
                        ):
    if clusterer is None:
        clusterer = BayesianGaussianMixture(n_init=5,
                                            n_components=n_clusters, 
                                            covariance_type="full",
                                            random_state=100)
    if reducer is None:
        reducer= None#PCA(len(organ_list),whiten=True)
    if fit_df is None:
        fit_df = sddf.copy()
    fit_vals = format_sddf(fit_df,features=features,organ_subset=organ_subset,normalize=normalize)
    if reducer is not None:
        reducer = reducer.fit(fit_vals)
        fit_vals = reducer.fit_transform(fit_vals)
            
    clusterer = clusterer.fit(fit_vals)
    def sddf_cluster_closure(tempdf):
        v = format_sddf(tempdf,features=features,organ_subset=organ_subset,normalize=normalize)
        if reducer is not None:
            v= reducer.transform(v)
        clusters = clusterer.predict(v)
        newdf = tempdf.copy()
        cname= prefix+'dose_clusters'
        newdf[cname] = clusters
        new_df = local_reorder_clusters(newdf,
                                  cname,
                                  by='mean_dose',
                                  organ_list=organ_subset#order by mean dose to clustered organs
                                 )
        score = clusterer.score_samples(v)
        newdf['fit_score'] = score
        return newdf[[prefix+'dose_clusters','fit_score']]
    return sddf_cluster_closure

def local_reorder_clusters(df,cname,by='mean_dose',organ_list=None):
    df = df.copy()
    df2 = df.copy()
    severities = {}
    clusts = sorted(df[cname].unique())
    getmean = lambda d: d[by].astype(float).mean()
    if organ_list is not None and Utils.iterable(df[by].iloc[0]):
        keep_idx = [Const.organ_list.index(o) for o in organ_list]
        df[by] = df[by].apply(lambda x: [x[i] for i in keep_idx])
    if Utils.iterable(df[by].iloc[0]):
        getmean = lambda d: np.stack(d[by].apply(lambda x: np.array(x).sum()).values).mean()
    for c in clusts:
        subset = df[df[cname] == c]
        avg_severity = getmean(subset)
        severities[c] = avg_severity
    clust_order = np.argsort(sorted(severities.keys(), key = lambda x: severities[x]))
    clust_map = {c: clust_order[i] for i,c in enumerate(clusts)}
    df2[cname] = df[cname].apply(lambda x: clust_map.get(x))
    return df2

class Clusterfier():
    
    outlier_ids = set([
        428,617,706,768,265,773,649,228,189,
    ])
    def __init__(self,
                 rds=None,
                 fit_df =None,
                 cluster_organs=None, 
                 n_clusters=3, 
                 cluster_features=None,
                 filter_outliers = False,
                 **kwargs,
                ):
        
        self.cluster_organs = cluster_organs if cluster_organs is not None else Const.organ_list[:]
        print(self.cluster_organs)
        self.n_clusters = n_clusters
        
        if cluster_features is None:
            cluster_features = ['V35','V40','V45','V50','V55','V60']
        self.cluster_features = cluster_features
        if fit_df is None:
            df = rds_to_df(rds,**kwargs)
        else:
            df = fit_df.copy()
        df = self.filter_df(df,filter_outliers= filter_outliers)
        self.df = df.copy()
        self.apply_clusters = get_cluster_closure(df,
                                                features=self.cluster_features,
                                                organ_subset=self.cluster_organs,
                                                n_clusters = self.n_clusters
                                                 )
        self.cluster_df=None
        self.current_mimic = None
        self.current_rules = None
        self.rule_candidates = None
        
        
    def filter_df(self,df,filter_outliers=False):
        keywords = ['_original','_max_','_6wk_symptoms','_late_symptoms']
        for keyword in keywords:
            to_drop = [col for col in df.columns if keyword in col]
            df = df.drop(to_drop,axis=1)
        if filter_outliers:
            iname = df.index.name
            df = df.reset_index()
            df = df[df.id.apply(lambda x: x not in ClusterTester.outlier_ids)]
            df = df.set_index(iname)
            print('after filter',df.shape)
        return df
    
cluster_args = [
    {
        'name': 'drymouth',
        'organs': [
        'Rt_Parotid_Gland',
        'Lt_Parotid_Gland',
        'Rt_Submandibular_Gland',
        'Lt_Submandibular_Gland',
        'Hard_Palate',
          ],
        'features': ['V25','V30','V35','V40','V45','V50','V55',"V60"]
    },
    {
        'name': 'voice',
        'organs': [
            'Tongue',
            'IPC',
            'Rt_Submandibular_Gland',
            'Larynx',
            'Supraglottic_Larynx'
        ],
        'features': ['V45','V50','V55','V60','V65'],
    },
     {
        'name': 'choke',
        'organs': [
               'IPC',
                'MPC',
                'Supraglottic_Larynx',
                "Esophagus",
                'Mylogeniohyoid_M',
            ],
        'features':  ['V30','V35','V40','V45','V50','V55','V60','V65'],
    },
     {
        'name': 'swallow',
        'organs': [
               'IPC',
                'MPC',
                'SPC',
                'Supraglottic_Larynx',
                'Esophagus',
            ],
        'features': ['V35','V40','V45','V50','V55','V60','V65','V70'],
    },
     {
        'name': 'mucus',
        'organs': [
                'Rt_Parotid_Gland','Lt_Parotid_Gland',
                'Rt_Submandibular_Gland','Lt_Submandibular_Gland',
            ],
        'features': ['V25','V30','V35','V40','V45','V50','V55','V60','V65'],
    },
     {
        'name': 'mucositis',
        'organs': [
                           'Hard_Palate',
                           'Soft_Palate',
                           'SPC',
                           'Genioglossus_M',
                           "Supraglottic_Larynx"
                       ],
        'features':  ['V40','V45','V50','V55','V60'],
    },
]

# clusterer = Clusterfier(rds=rds, cluster_organs = cluster_args[0]['organs'],cluster_features=cluster_args[1]['features'],required=['M6_mdasi_drymouth'])
# clusterer.apply_clusters(dose_df.copy())

cluster_results = pd.DataFrame(dose_df.index).set_index('id')
for cargs in cluster_args:
    name = cargs['name']
    clusterer_filtered = Clusterfier(
        rds = rds,
        cluster_organs = cargs['organs'],
        cluster_features=cargs['features'],
        required=['M6_mdasi_'+name]
    )
    c1 = clusterer_filtered.apply_clusters(dose_df.copy())
    c1 = c1.rename({'dose_clusters':'dose_clusters_'+name+'_filtered','fit_score': name+'filtered_fit_score'},axis=1)
    cluster_results = cluster_results.merge(c1,left_index=True,right_index=True)
    
    clusterer_unfiltered = Clusterfier(
        fit_df=dose_df,
        cluster_organs = cargs['organs'],
        cluster_features=cargs['features'],
    )
    c2 = clusterer_unfiltered.apply_clusters(dose_df.copy())
    c2 = c2.rename({'dose_clusters':'dose_clusters_'+name+'_unfiltered','fit_score': name+'_unfiltered_fit_score'},axis=1)
    cluster_results = cluster_results.merge(c2,left_index=True,right_index=True)
cluster_results

['Rt_Parotid_Gland', 'Lt_Parotid_Gland', 'Rt_Submandibular_Gland', 'Lt_Submandibular_Gland', 'Hard_Palate']
before drop count 823
after drop count 511
error (%) 0.7169430029265533
[('severe_6wk_symptoms', 0.011741682974559686), ('severe_late_symptoms', 0.01761252446183953), ('moderate_6wk_symptoms', 0.05283757338551859), ('moderate_late_symptoms', 0.05870841487279843), ('mild_6wk_symptoms', 0.20156555772994128), ('mild_late_symptoms', 0.20743639921722112)]
dose ids: 484 syptom ids: 511 merged ids: 349
['Rt_Parotid_Gland', 'Lt_Parotid_Gland', 'Rt_Submandibular_Gland', 'Lt_Submandibular_Gland', 'Hard_Palate']
['Tongue', 'IPC', 'Rt_Submandibular_Gland', 'Larynx', 'Supraglottic_Larynx']
before drop count 823
after drop count 508
error (%) 0.6599186271873672
[('severe_6wk_symptoms', 0.00984251968503937), ('severe_late_symptoms', 0.015748031496062992), ('moderate_6wk_symptoms', 0.051181102362204724), ('moderate_late_symptoms', 0.05511811023622047), ('mild_6wk_symptoms', 0.19881889763779528),

,dose_clusters_drymouth_filtered,drymouthfiltered_fit_score,dose_clusters_drymouth_unfiltered,drymouth_unfiltered_fit_score,dose_clusters_voice_filtered,voicefiltered_fit_score,dose_clusters_voice_unfiltered,voice_unfiltered_fit_score,dose_clusters_choke_filtered,chokefiltered_fit_score,...,dose_clusters_swallow_unfiltered,swallow_unfiltered_fit_score,dose_clusters_mucus_filtered,mucusfiltered_fit_score,dose_clusters_mucus_unfiltered,mucus_unfiltered_fit_score,dose_clusters_mucositis_filtered,mucositisfiltered_fit_score,dose_clusters_mucositis_unfiltered,mucositis_unfiltered_fit_score
id,,,,,,,,,,,,,,,,,,,,,
2,2,54.319103,0,54.550151,0,6.847194,0,5.822175,1,8.657162,...,1,8.072902,0,49.413380,2,52.326469,0,18.330475,2,18.721286
6,2,50.426105,1,46.610483,0,5.601179,0,8.378759,1,14.951386,...,1,22.514302,0,47.218463,2,52.974053,1,27.053132,0,28.234137
7,2,53.889922,1,49.980984,0,-3.724318,0,-3.757568,1,2.852160,...,1,17.986111,0,52.353040,2,54.744649,0,7.454658,2,7.109430
8,2,53.516441,1,49.574101,0,1.916884,0,2.507787,0,18.850006,...,1,16.058448,0,50.099761,2,52.611435,1,23.639095,0,24.318256
9,1,50.250637,0,48.465040,1,2.769099,1,5.871392,2,18.045581,...,2,24.394512,1,56.929853,0,59.595305,2,19.485470,0,22.485722
10,2,55.603159,1,55.136618,2,4.912969,2,7.853782,0,26.651914,...,0,28.278006,0,51.529484,2,54.327072,0,16.408110,2,16.374277
11,2,7.066488,0,15.846014,1,-110.658252,0,-62.275208,1,-85.062334,...,1,2.624206,0,1.084286,2,23.053589,0,-43.324644,2,-14.021781
12,1,54.985707,0,53.144495,1,24.011941,1,30.546345,2,53.142817,...,2,61.378585,1,55.738985,0,58.199553,2,21.959182,1,13.813521
13,2,53.206213,0,48.856097,0,3.983861,0,3.064071,1,15.764841,...,1,16.076300,0,44.745006,2,48.404325,1,25.664034,0,25.739506


In [77]:
cluster_results.to_csv(Const.data_dir + 'mdasi_dose_clusters.csv')